### Skew-T Radiosonde plot
\$ source activate py36skewT

\$ jupyter notebook


http://pythonhosted.org/pymeteo/#plotting

https://github.com/cwebster2/pyMeteo

http://kbkb-wx-python.blogspot.no/2015/07/plotting-sounding-data-from-university.html

In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Applications/pyMeteo/pymeteo/')
import numpy as np
import createFolder as cF
import netCDF4
import fill_values as fv

from datetime import date
import datetime
import pandas as pd


import plot_skewT as skewT
import thermo
from pymeteo.constants import *
import calc_date as cd


In [ ]:
savefig = 0

stn = 'Haukeliseter' #
year= '2016'
month = '12'

hh = '00'

cF.createFolder('../../Figures/Sounding/MEPS/%s/_%s' %(stn,hh))      # for figure files
t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
#t = ['22']


In [ ]:
var_name_T = 'air_temperature_ml'
var_name_u = 'x_wind_ml'
var_name_v = 'y_wind_ml'
var_name_qv = 'specific_humidity_ml'

In [ ]:
#file_2m    = dict()
T_z_p_file = dict()
u_file     = dict()
v_file     = dict()
qv_file    = dict()

#fn_2m      = dict()
fn_T_z_p   = dict()
fn_u       = dict()
fn_v       = dict()
fn_qv      = dict()

time         = dict()
T            = dict()
p            = dict()
thickness_ml = dict()
u            = dict()
v            = dict()
height_ml    = dict()
height       = dict()
qv           = dict()
Td           = dict()

In [ ]:
for ini_day in t:

    for ens_memb in range(0,10):
###### OPEN  files ######################
# open file with 
#    file_2m[ens_memb]    = '../../Data/MEPS/%s/2m_00/%s%s%s_%s_%s.nc'    %(stn,year,month,day,hh,ens_memb)
        T_z_p_file[ens_memb] = '../../Data/MEPS/%s/ml_00/%s/%s%s%s_00_%s.nc' %(stn,var_name_T,year,month,ini_day,ens_memb)
        u_file[ens_memb]     = '../../Data/MEPS/%s/ml_00/%s/%s%s%s_00_%s.nc' %(stn,var_name_u,year,month,ini_day,ens_memb)
        v_file[ens_memb]     = '../../Data/MEPS/%s/ml_00/%s/%s%s%s_00_%s.nc' %(stn,var_name_v,year,month,ini_day,ens_memb)
        qv_file[ens_memb]    = '../../Data/MEPS/%s/ml_00/%s/%s%s%s_00_%s.nc' %(stn,var_name_qv,year,month,ini_day,ens_memb)
    
#    fn_2m[ens_memb]      = netCDF4.Dataset(file_2m[ens_memb])
        fn_T_z_p[ens_memb]   = netCDF4.Dataset(T_z_p_file[ens_memb])
        fn_u[ens_memb]       = netCDF4.Dataset(u_file[ens_memb])
        fn_v[ens_memb]       = netCDF4.Dataset(v_file[ens_memb])
        fn_qv[ens_memb]      = netCDF4.Dataset(qv_file[ens_memb])

# Read in time to know initialisation time (plotting)    
        time[ens_memb] = fn_T_z_p[ens_memb].variables['time']
        time[ens_memb] = fv.fill_nan(time[ens_memb][:])
        

    

# Read in the variable names which are needed for plotting
        T[ens_memb] = fn_T_z_p[ens_memb].variables[var_name_T]   


        p[ens_memb] = fn_T_z_p[ens_memb].variables['pressure_ml'] 
        u[ens_memb] = fn_u[ens_memb].variables[var_name_u]
        v[ens_memb] = fn_v[ens_memb].variables[var_name_v]
        qv[ens_memb]= fn_qv[ens_memb].variables[var_name_qv]
    
# combine with 2m obs   
 #   T2  = fn_2m[0].variables['air_temperature_2m']
  #  qv2 = fn_2m[0].variables['specific_humidity_2m']
#    TT2 = pd.DataFrame.from_dict(T2[:,:])
 #   TTml = pd.DataFrame.from_dict(T[0][:,:])
  #  TTall = np.asarray(pd.concat([TT2,TTml],axis = 1))
    
# layer thickness to calculate height
        thickness_ml[ens_memb]      = fn_T_z_p[ens_memb].variables['layer_thickness']
        thickness_ml[ens_memb]      = fv.fill_nan(thickness_ml[ens_memb][:])
    
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        th_arr = np.nansum([np.asarray(h_above)[:,0], thickness[:,0]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i-1]], axis = 0)
            h_above.append(th_arr)
        height_ml[ens_memb] = np.transpose(h_above)


        
# substitute missing values with nan       
        T[ens_memb]    = fv.fill_nan(T[ens_memb][:])
        p[ens_memb]    = fv.fill_nan(p[ens_memb][:])
        u[ens_memb]    = fv.fill_nan(u[ens_memb][:])
        v[ens_memb]    = fv.fill_nan(v[ens_memb][:])
        qv[ens_memb]   = fv.fill_nan(qv[ens_memb][:])


# calculate dew point with thermo.py
        Td[ens_memb] = thermo.Td(p[ens_memb],qv[ens_memb])
    
# get the temperature in °C
        T[ens_memb] = T[ens_memb]-T00
        Td[ens_memb] = Td[ens_memb]-T00




# Read in time to know initialisation time (plotting)    
    day = datetime.datetime.utcfromtimestamp(time[0][int(hh)]).day   # day of initialisation
    hour = datetime.datetime.utcfromtimestamp(time[0][int(hh)]).hour       # first hour of initialisation?
    calday, calmon = cd.get_dayname(year, month, day)

### plot vertical temp profile
    if hour < 10:
        hour = '0%s' %hour
    
    figname = '../../Figures/Sounding/%s/%s%s%s_%s.png' %(stn,year,month,ini_day,hh)
    title = '%s, %s %s %s   %s$\,$UTC' %(calday, day, calmon, year, hour)

    
    skewT.plot_skewT_EM(T, Td, height_ml, p, u, v, int(hh),title, savefig, figname)
    print('saved: %s' %figname)